In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
superstore = pd.read_excel('Global Superstore (1).xlsx')
superstore.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,City,...,Product ID,Category,SubCategory,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [3]:
data = superstore.loc[:,['Row ID','Order ID','City','SubCategory','Product Name','Quantity']]
data.head()

,Row ID,Order ID,City,SubCategory,Product Name,Quantity
0,40098,CA-2014-AB10015140-41954,Oklahoma City,Phones,Samsung Convoy 3,2
1,26341,IN-2014-JR162107-41675,Wollongong,Chairs,"Novimex Executive Leather Armchair, Black",9
2,25330,IN-2014-CR127307-41929,Brisbane,Phones,"Nokia Smart Phone, with Caller ID",9
3,13524,ES-2014-KM1637548-41667,Berlin,Phones,"Motorola Smart Phone, Cordless",5
4,47221,SG-2014-RH9495111-41948,Dakar,Copiers,"Sharp Wireless Fax, High-Speed",8


In [4]:
data.isnull().sum()

Row ID          0
Order ID        0
City            0
SubCategory     0
Product Name    0
Quantity        0
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Row ID        51290 non-null  int64 
 1   Order ID      51290 non-null  object
 2   City          51290 non-null  object
 3   SubCategory   51290 non-null  object
 4   Product Name  51290 non-null  object
 5   Quantity      51290 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.3+ MB


In [6]:
data.dropna(axis=0, subset=['Order ID'], inplace=True)

In [7]:
data['City'].value_counts()

New York City     915
Los Angeles       747
Philadelphia      537
San Francisco     510
Santo Domingo     443
                 ... 
Caraguatatuba       1
Arad                1
Tatuí               1
Maidstone           1
Victoria Falls      1
Name: City, Length: 3650, dtype: int64

In [32]:
basket = (data[data['City'] =="New York City"]
          .groupby(['Order ID', 'SubCategory'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Order ID'))
basket.head()

SubCategory,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
Order ID,,,,,,,,,,,,,,,,,
CA-2012-AA10315140-41166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2012-AG10270140-41112,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2012-AG10390140-41154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2012-AH10030140-41273,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2012-AJ10960140-41272,0.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
def encode_units(x):
    if x <= 0:
        return bool(0)
    if x >= 1:
        return bool(1)

basket_sets = basket.applymap(encode_units)
basket_sets.head()

SubCategory,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
Order ID,,,,,,,,,,,,,,,,,
CA-2012-AA10315140-41166,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False
CA-2012-AG10270140-41112,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
CA-2012-AG10390140-41154,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
CA-2012-AH10030140-41273,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
CA-2012-AJ10960140-41272,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False


In [34]:
frequent_itemsets = apriori(basket_sets,min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.126667,(Accessories)
1,0.077778,(Appliances)
2,0.144444,(Art)
3,0.277778,(Binders)
4,0.060000,(Bookcases)
...,...,...
85,0.013333,"(Storage, Binders, Phones)"
86,0.011111,"(Chairs, Storage, Phones)"
87,0.011111,"(Furnishings, Phones, Paper)"
88,0.011111,"(Furnishings, Storage, Paper)"


In [35]:
rules = association_rules(frequent_itemsets,metric="lift")

In [36]:
rules.head(100)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Art),(Accessories),0.144444,0.126667,0.015556,0.107692,0.850202,-0.002741,0.978736
1,(Accessories),(Art),0.126667,0.144444,0.015556,0.122807,0.850202,-0.002741,0.975333
2,(Binders),(Accessories),0.277778,0.126667,0.035556,0.128000,1.010526,0.000370,1.001529
3,(Accessories),(Binders),0.126667,0.277778,0.035556,0.280702,1.010526,0.000370,1.004065
4,(Chairs),(Accessories),0.128889,0.126667,0.017778,0.137931,1.088929,0.001452,1.013067
...,...,...,...,...,...,...,...,...,...
95,(Paper),(Tables),0.235556,0.046667,0.015556,0.066038,1.415094,0.004563,1.020741
96,(Storage),(Phones),0.157778,0.177778,0.028889,0.183099,1.029930,0.000840,1.006513
97,(Phones),(Storage),0.177778,0.157778,0.028889,0.162500,1.029930,0.000840,1.005638
98,(Storage),(Tables),0.157778,0.046667,0.011111,0.070423,1.509054,0.003748,1.025556


In [37]:
rules[ (rules['lift'] >= 3) &
       (rules['confidence'] >= 0.3) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
100,"(Fasteners, Binders)",(Accessories),0.031111,0.126667,0.015556,0.500000,3.947368,0.011615,1.746667
101,"(Binders, Accessories)",(Fasteners),0.035556,0.084444,0.015556,0.437500,5.180921,0.012553,1.627654
102,"(Fasteners, Accessories)",(Binders),0.015556,0.277778,0.015556,1.000000,3.600000,0.011235,inf
124,"(Storage, Phones)",(Accessories),0.028889,0.126667,0.011111,0.384615,3.036437,0.007452,1.419167
126,"(Phones, Accessories)",(Storage),0.022222,0.157778,0.011111,0.500000,3.169014,0.007605,1.684444
130,"(Fasteners, Binders)",(Chairs),0.031111,0.128889,0.013333,0.428571,3.325123,0.009323,1.524444
131,"(Binders, Chairs)",(Fasteners),0.026667,0.084444,0.013333,0.500000,5.921053,0.011081,1.831111
142,"(Storage, Binders)",(Fasteners),0.046667,0.084444,0.015556,0.333333,3.947368,0.011615,1.373333
143,"(Fasteners, Binders)",(Storage),0.031111,0.157778,0.015556,0.500000,3.169014,0.010647,1.684444
178,"(Storage, Chairs)",(Phones),0.017778,0.177778,0.011111,0.625000,3.515625,0.007951,2.192593


In [38]:
Output=round(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']],3) 

In [39]:
Output

,antecedents,consequents,support,confidence,lift
0,(Art),(Accessories),0.016,0.108,0.850
1,(Accessories),(Art),0.016,0.123,0.850
2,(Binders),(Accessories),0.036,0.128,1.011
3,(Accessories),(Binders),0.036,0.281,1.011
4,(Chairs),(Accessories),0.018,0.138,1.089
...,...,...,...,...,...
197,"(Labels, Paper)",(Storage),0.013,0.545,3.457
198,"(Storage, Paper)",(Labels),0.013,0.462,6.294
199,(Labels),"(Storage, Paper)",0.013,0.182,6.294
200,(Storage),"(Labels, Paper)",0.013,0.085,3.457


In [42]:
targeted_rules=Output[Output["consequents"] == {'Paper'}]
print(targeted_rules)

                antecedents consequents  support  confidence   lift
13            (Accessories)     (Paper)    0.036       0.281  1.192
44                (Binders)     (Paper)    0.056       0.200  0.849
54              (Bookcases)     (Paper)    0.018       0.296  1.258
64                 (Chairs)     (Paper)    0.027       0.207  0.878
74              (Fasteners)     (Paper)    0.018       0.211  0.894
80            (Furnishings)     (Paper)    0.044       0.290  1.231
86                 (Labels)     (Paper)    0.024       0.333  1.415
92                 (Phones)     (Paper)    0.036       0.200  0.849
94                 (Tables)     (Paper)    0.016       0.333  1.415
107  (Binders, Accessories)     (Paper)    0.011       0.312  1.327
119   (Phones, Accessories)     (Paper)    0.011       0.500  2.123
136       (Binders, Chairs)     (Paper)    0.011       0.417  1.769
148  (Furnishings, Binders)     (Paper)    0.016       0.368  1.564
166       (Binders, Phones)     (Paper)    0.016

In [46]:
def recommender(rules, product_name, rec_count=1):
    sorted_rules = rules.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_name:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

# When we sort the items in the cart by Lift,
# we can select the desired product and see the desired number of other products that are most closely related to it.


# For example; 'Paper'
recommender(rules, ( 'Paper'), 5) 

['Labels', 'Labels', 'Storage', 'Furnishings', 'Accessories']